In [19]:
# Setup
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load Data
DATA_DIR = '../data/spectogram/'
X, y = [], []

def add_noise(spectrogram, noise_level=0.02):
    noise = np.random.normal(0, noise_level, spectrogram.shape)
    return np.clip(spectrogram + noise, 0, 1)

def time_mask(spec, mask_size=10):
    spec = spec.copy()
    t = np.random.randint(0, spec.shape[1] - mask_size)
    spec[:, t:t+mask_size] = 0
    return spec

# Load spectrograms and augment
for label, folder in enumerate(['nonhuman', 'human']):
    folder_path = os.path.join(DATA_DIR, folder)
    for file in os.listdir(folder_path):
        if file.endswith('.npy'):
            spectrogram = np.load(os.path.join(folder_path, file))
            if np.max(spectrogram) > -100 and not np.isnan(spectrogram).any():
                spectrogram = (spectrogram + 80) / 80  # normalize to [0,1]
                X.append(spectrogram)
                y.append(label)

                # Augmentations
                X.append(add_noise(spectrogram, noise_level=0.03))
                y.append(label)
                X.append(time_mask(spectrogram))
                y.append(label)

X = np.array(X).reshape(-1, 64, 64, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Loaded data train: {X_train.shape}, Test: {X_test.shape}")

Loaded data train: (14486, 64, 64, 1), Test: (3622, 64, 64, 1)


In [20]:
import numpy as np

print("Train label distribusi:")
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

print("Test label distribusi:")
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

print(f"Train min: {X_train.min()}, max: {X_train.max()}")
print(f"Test min: {X_test.min()}, max: {X_test.max()}")

print(np.bincount(y_train))
print(np.bincount(y_test))

Train label distribusi:
{0: 5397, 1: 9089}
Test label distribusi:
{0: 1350, 1: 2272}
(array([0, 1]), array([5397, 9089], dtype=int64))
(array([0, 1]), array([1350, 2272], dtype=int64))
Train min: 0.0, max: 1.0
Test min: 0.0, max: 1.0
[5397 9089]
[1350 2272]


In [21]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.BatchNormalization(),

    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

class_weights = dict(enumerate(
    class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
))

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=64,
    epochs=50,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 121s 505ms/step - accuracy: 0.6406 - loss: 0.6414 - val_accuracy: 0.4818 - val_loss: 0.7145 - learning_rate: 5.0000e-04
Epoch 2/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 118s 400ms/step - accuracy: 0.7931 - loss: 0.4218 - val_accuracy: 0.5204 - val_loss: 0.6990 - learning_rate: 5.0000e-04
Epoch 3/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 90s 397ms/step - accuracy: 0.8655 - loss: 0.2967 - val_accuracy: 0.6438 - val_loss: 0.7054 - learning_rate: 5.0000e-04
Epoch 4/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 92s 405ms/step - accuracy: 0.8882 - loss: 0.2485 - val_accuracy: 0.8840 - val_loss: 0.2664 - learning_rate: 5.0000e-04
Epoch 5/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 89s 390ms/step - accuracy: 0.9123 - loss: 0.1961 - val_accuracy: 0.7885 - val_loss: 0.5353 - learning_rate: 5.0000e-04
Epoch 6/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 90s 395ms/step - accuracy: 0.9349 - loss: 0.1523 - val_accuracy: 0.8539 - val_loss: 0.3627 - learning_rate: 5.0000e-04
Epoch 7/50
227/227 ━━━━━━━━━━━━━━━━━━━━ 95s 

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = model.predict(X_test)

y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred, digits=4))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

114/114 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step
              precision    recall  f1-score   support

           0     0.9711    0.9696    0.9703      1350
           1     0.9820    0.9828    0.9824      2272

    accuracy                         0.9779      3622
   macro avg     0.9765    0.9762    0.9764      3622
weighted avg     0.9779    0.9779    0.9779      3622

Confusion Matrix:
[[1309   41]
 [  39 2233]]


In [24]:
model.save('../models/model.h5')
print("Model saved to '../models/model.h5'")

Model saved to '../models/model.h5'


In [25]:
import tensorflow as tf

model = tf.keras.models.load_model('../models/model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('../models/model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model TFLite saved to '../models/model.tflite'")

INFO:tensorflow:Assets written to: C:\Users\Asus\AppData\Local\Temp\tmpqol9pa80\assets


INFO:tensorflow:Assets written to: C:\Users\Asus\AppData\Local\Temp\tmpqol9pa80\assets


Saved artifact at 'C:\Users\Asus\AppData\Local\Temp\tmpqol9pa80'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2023937677200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023937676048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023937687376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917454928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023937676624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023937683920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917455888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917456464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917456656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917457232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2023917